In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .appName("AlsModel") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .getOrCreate()


25/07/22 18:59:08 WARN Utils: Your hostname, vaibhavi-HP-Laptop-15-fd0xxx resolves to a loopback address: 127.0.1.1; using 192.168.0.128 instead (on interface wlo1)
25/07/22 18:59:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/22 18:59:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/07/22 18:59:13 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
df = spark.read.parquet("/home/vaibhavi/spark-ml-venv/ml_project/preprocessing/output/als/*")

In [4]:
df.head()

Row(User_id='A01254073JW8SSTKH6AIB', Id='0451521196', rating=5.0)

In [5]:
df.printSchema()

root
 |-- User_id: string (nullable = true)
 |-- Id: string (nullable = true)
 |-- rating: double (nullable = true)



In [6]:
# as we can see there are no null values, but the als model needs only numeric data for the clculation, so converting all the user-id and id into numeric:
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, sum

# String index userId and itemId (which may both be strings)
user_indexer = StringIndexer(inputCol="User_id", outputCol="userIdx", handleInvalid="skip")


# Fit and transform
df_indexed = user_indexer.fit(df).transform(df)

# Cast to int
df_ready = df_indexed.select(
    col("userIdx").cast("int").alias("user"),
    col("Id").cast("int").alias("id"),
    col("rating").cast("float")
)


In [7]:
df_top = spark.read.parquet("/home/vaibhavi/spark-ml-venv/ml_project/book_recommender/data/top_users.parquet")


In [8]:
df_top = df_top.select("user").limit(100)

In [9]:
df_top.count()

100

In [10]:
from pyspark.sql.functions import col

top_user_ratings = df_top.join(df_ready, on="user", how="inner")


In [11]:
top_user_ratings.printSchema()

root
 |-- user: integer (nullable = true)
 |-- id: integer (nullable = true)
 |-- rating: float (nullable = true)



In [12]:
#storing this for the training als:
top_user_ratings.write.mode("overwrite").parquet("data/top_user_ratings.parquet")


25/07/22 19:00:22 WARN DAGScheduler: Broadcasting large task binary with size 38.3 MiB


In [13]:
top_user_ratings.count()

25/07/22 19:01:01 WARN DAGScheduler: Broadcasting large task binary with size 38.1 MiB


62439

In [2]:
from pyspark.sql.functions import col, sum as Fsum

top_user_ratings.select([Fsum(col(c).isNull().cast("int")).alias(c) for c in df.columns]).show()

NameError: name 'top_user_ratings' is not defined